# <font color="red">CMPT318 PROJECT</font>
MEMBER:
- JEFF
- KELVIN
- JACKY

In [1]:
import pandas as pd
import numpy as np
from scipy import misc
from skimage.io import imread_collection
import re
import glob

## <font color="blue"> I. LOAD THE DATA </font>

### i. Load Webcam Images

In [2]:
image_collection = imread_collection('katkam-scaled/*.jpg')
images_np = np.array(image_collection)

In [3]:
# # IMAGE IN NUMPY ARRAY.
# images_np
# images_np.shape

In [4]:
images_df = pd.DataFrame(
    images_np.reshape(images_np.shape[0],images_np.shape[1] * images_np.shape[2] * images_np.shape[3]))

In [5]:
# IMAGE IN PANDAS DATAFRAME, AND BEEN RESHAPED.
# images_df
# images_df.shape

## ii. Load Weather Observations Data

In [6]:
#reference: for load multiple file from forder: https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe
weather_obv_filenames = glob.glob('yvr-weather/*.csv')
weather_obv_df = []

for filename in weather_obv_filenames:
    weather_obv_df.append(pd.read_csv(filename, skiprows=15)) # first 15 rows are general information, which is not useful data.

weather_obv_df = pd.concat(weather_obv_df).reset_index()
# pd.read_csv('weather-51442-201607.csv', skiprows=15)

In [7]:
# WEATHER OBSERVATIONS DATAFRAME.
# weather_obv_df

## <font color="blue">II. CLEAN&PREPARE DATA</font>

### i. Clean Webcam Images

In [8]:
# GET IMAGE FILE_NAME
image_filenames = np.array(image_collection.files)
# image_filenames[0]

In [9]:
# extract image shoot date form filename
re_image_date = re.compile(r'katkam-\d\d\d\d\d\d\d\d\d\d\d\d\d\d')
def get_image_time(path):
    matches = re_image_date.findall(path)
    if matches:
        # preocess match learned from https://docs.python.org/2/library/re.html
        # take the last match which will be file name
        result = matches[-1]
        return (int(result[-14:-10]), int(result[-10:-8]), int(result[-8:-6]), result[-6:-4] + ':' + result[-4:-2])
    else:
        return "wrong input file name format"
get_image_time = np.vectorize(get_image_time)
images_date = get_image_time(image_filenames)

In [10]:
# EXTRACTED IMAGE DATE
# images_date

In [11]:
# add relevant columns
images_df['Year'] = images_date[0]
images_df['Month'] = images_date[1]
images_df['Day'] = images_date[2]
images_df['Time'] = images_date[3]

In [12]:
# IMAGE DATA WITH DATE TIME
# images_df['Time']

## ii. Clean Weather Observations Data

In [13]:
# DROP UNCESSARY COLUMNS
cleaned_weather_obv = weather_obv_df.drop(['index','Data Quality'], axis=1)
cleaned_weather_obv = cleaned_weather_obv.drop(['Temp Flag', 'Stn Press Flag','Wind Chill Flag', 'Hmdx Flag', 'Visibility Flag', 'Wind Spd Flag', 'Wind Dir Flag', 'Rel Hum Flag', 'Dew Point Temp Flag'], axis=1)

# type(weather_obv_df['Year'][0])

In [14]:
# SPLIT cleaned_weather INTO TWO DATAFRAME, ONE WITH NaN WEATHER COLUMN, ONE WITH NOT NaN COLUMN
data_whoseWeather_IsNaN = cleaned_weather_obv[~ cleaned_weather_obv.Weather.notnull()] # weather with Nan
main_training_data = cleaned_weather_obv[cleaned_weather_obv.Weather.notnull()] # weather without Nan

In [15]:
# TWO SPLIT DATAFRAME
# data_whoseWeather_IsNaN
# main_training_data

In [16]:
main_training_data_withoutHW = main_training_data.drop(['Hmdx', 'Wind Chill'],axis=1)
final_data = main_training_data_withoutHW.dropna().copy()
# main_training_data = cleaned_data.copy()

In [17]:
# final_data

### weather description category include
- Clear: Clear, Mainly Clear
- Cloudy: Cloudy, Mostly Cloudy	
- Fog: Fog, Freezing Fog,
- Rain: Drizzle, Freezing Rain, Heavy Rain, Moderate Rain, Moderate Rain Showers, Rain, Rain Showers, Thunderstorms	
- Snow: Moderate Snow, Snow Pellets	,Ice Pellets, Snow Showers

<font color=red>Jacky留言：可以回头讨论 要怎么分类， 我暂时分成不止这5类， 回头商讨商讨</font>

In [18]:
# CLEAN HUMAN-GENERATED WEATHER DESCRIPTION
# SEE THE TOTAL VARIOUS WEATHER DESCRIPTION USE GROUPBY
# weather_category = final_data.groupby('Weather').count()
# weather_category

In [19]:
# #for reduce class
# sthis output string
def removeDuplicate(lst):
    lst = lst.split(",")
#     print(lst)
    newlst = ""
    for i in lst:
#         print (i)
        if i not in newlst:
            newlst = newlst + i + ','
    return newlst




# # removeDuplicate("Rain,Clear,Rain,Rain,Fog") for test

In [20]:
# re_clear = re.compile(r'Clear')
# re_cloudy = re.compile(r'Cloudy')
# re_fog = re.compile(r'Fog')
# re_rain = re.compile(r'Rain')
# re_snow = re.compile(r'Snow')
# re_drizzle = re.compile(r'Drizzle')
# re_thunderstorms = re.compile(r'Thunderstorms')
# re_ice = re.compile(r'Ice')

# def clean_weather_description(Str):
#     result = ''
#     match_clear = re_clear.search(Str)
#     if match_clear:
#         result = result + match_clear[0]+','
        
#     match_cloudy = re_cloudy.search(Str)
#     if match_cloudy:
#         result = result + match_cloudy[0]+','
        
#     match_fog = re_fog.search(Str)
#     if match_fog:
#         result = result + match_fog[0]+','
        
#     match_rain = re_rain.search(Str)
#     if match_rain:
#         result = result + match_rain[0]+','
        
#     match_snow = re_snow.search(Str)
#     if match_snow:
#         result = result + match_snow[0]+','
        
#     match_drizzle = re_drizzle.search(Str)
#     if match_drizzle:
#         result = result + match_drizzle[0]+','
        
#     match_thunderstorms = re_thunderstorms.search(Str)
#     if match_thunderstorms:
#         result = result + match_thunderstorms[0]+','
        
#     match_ice = re_ice.search(Str)
#     if match_ice:
#         result = result + match_ice[0]+','
        
#     return result[:-1]

re_clear = re.compile(r'Clear')
re_cloudy = re.compile(r'Cloudy')
re_fog = re.compile(r'Fog')
re_rain = re.compile(r'Rain')
re_snow = re.compile(r'Snow')
re_drizzle = re.compile(r'Drizzle')
re_thunderstorms = re.compile(r'Thunderstorms')
re_ice = re.compile(r'Ice')

# this output string
def clean_weather_description(Str):
    result = ''
    match_clear = re_clear.search(Str)
    if match_clear:
        result = result + match_clear[0]+','
        
    match_cloudy = re_cloudy.search(Str)
    if match_cloudy:
        result = result + match_cloudy[0]+','
        
    match_fog = re_fog.search(Str)
    if match_fog:
        result = result + match_fog[0]+','
        
    match_rain = re_rain.search(Str)
    if match_rain:
        result = result + match_rain[0]+','
        
    match_snow = re_snow.search(Str)
    if match_snow:
        result = result + match_snow[0]+','
        
    match_drizzle = re_drizzle.search(Str)
    if match_drizzle:
        result = result + 'Rain'+','
        
    match_thunderstorms = re_thunderstorms.search(Str)
    if match_thunderstorms:
        result = result + 'Rain'+','
        
    match_ice = re_ice.search(Str)
    if match_ice:
        result = result + 'Snow'+','
    
    result = removeDuplicate(result)
        
    return result[:-1]



# clean_weather_description('Thunderstorms,Rain Showers')#test the function


In [21]:
# CLEAN THE WEATHER DESCRIPTION CATEGORY
final_data['Weather'] =final_data['Weather'].apply(clean_weather_description)

In [22]:
# final_data[final_data['Weather'] == 'Rain']
# SHOW THE NEW WEATHER CATEGORY
# final_data
# weather_category = final_data.groupby('Weather').count()
# weather_category

## iii. Join Cleaned Webcam Image and Cleaned Weather obserbations Data Together

In [23]:
merged_data = final_data.merge(right = images_df, on = ['Year', 'Month', 'Day', 'Time'], how = 'inner')

In [24]:
merged_data.iloc[:,13:]

,0,1,2,3,4,5,6,7,8,9,...,147446,147447,147448,147449,147450,147451,147452,147453,147454,147455
0,89,121,180,89,121,180,89,121,180,89,...,0,0,0,0,0,0,0,0,0,0
1,125,150,191,122,147,188,121,146,187,122,...,0,0,0,0,0,0,0,0,0,0
2,131,160,200,132,161,201,134,163,203,136,...,0,0,0,0,0,0,0,0,0,0
3,183,210,239,183,210,239,184,211,240,184,...,0,0,0,0,0,0,0,0,0,0
4,140,148,151,141,149,152,143,151,154,144,...,0,0,0,0,0,0,0,0,0,0
5,119,151,198,119,151,198,119,151,198,119,...,0,0,0,0,0,0,0,0,0,0
6,85,114,174,84,113,173,83,112,172,82,...,0,0,0,0,0,0,0,0,0,0
7,138,164,189,139,165,190,142,168,193,144,...,0,0,0,0,0,0,0,0,0,0
8,157,189,228,157,189,228,157,189,228,157,...,0,0,0,0,0,0,0,0,0,0
9,115,122,132,115,122,132,115,122,132,116,...,0,0,0,0,0,0,0,0,0,0


## <font color="blue"> III. ANALYSE THE DATA </font>

## <font color="black">Split train_test data</font>

In [25]:
from sklearn.model_selection import train_test_split
X = merged_data.iloc[:,13:]
y = merged_data['Weather']
# bayes_X_train, bayes_X_test, bayes_y_train, bayes_y_test = train_test_split(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y)


## <font color="black">try Bayesian Classifier</font>

In [26]:
# from sklearn.naive_bayes import GaussianNB
# model = GaussianNB()
# # model.fit(bayes_X_train, bayes_y_train)
# # print(model.score(bayes_X_test, bayes_y_test))
# model.fit(X_train, y_train)
# print(model.score(X_test, y_test)) #within 1 min

In [27]:
# from scipy import stats
# print(stats.normaltest(X_train).pvalue)
# # p-value too small no-normal
# # don't use it

## <font color="black">SVC</font>

In [28]:
# from sklearn.svm import SVC
# # from sklearn.model_selection import train_test_split
# # X_train, X_test, y_train, y_test = train_test_split(X, y)
# model = SVC(kernel='linear')
# model.fit(X_train, y_train)
# print(model.score(X_test, y_test))  #57 0.622103386809

In [29]:
# model = SVC(kernel='rbf', decision_function_shape='ovr')     score = 0.397504456328   around 23min
# model = SVC(kernel='linear')        score = 0.668449197861   / 0.620320855615 ///0.641711229947
# model = SVC(kernel='rbf', decision_function_shape='ovr')   score = 0.623885918004
# e-1 model = SVC(kernel='linear', C=1e-1) 41   0.620320855615

## <font color="black">PCA + SVC</font>

In [30]:
# from sklearn.pipeline import make_pipeline
# from sklearn.decomposition import PCA
# model = make_pipeline(
#     PCA(1000),
#     SVC(kernel='linear', C=2.0)
# )
# model.fit(X_train, y_train)
# print(model.score(X_test, y_test))   #0.614973262032/0.620320855615   37-39 2 min fast 18-20

In [31]:
# from sklearn.pipeline import make_pipeline
# from sklearn.decomposition import PCA
# model = make_pipeline(
#     PCA(1000),
#     SVC(kernel='linear', C=0.1)
# )
# model.fit(X_train, y_train)
# print(model.score(X_test, y_test)) # little bit better 0.625668449198 / 0.6096256684492.03min

In [32]:
# from sklearn.pipeline import make_pipeline
# from sklearn.decomposition import PCA
# model = make_pipeline(
#     PCA(100),
#     SVC(kernel='linear', C=2.0)
# )
# model.fit(X_train, y_train)
# print(model.score(X_test, y_test)) #02 - 36  very slow, why????? around 35 min

In [33]:
# from sklearn.pipeline import make_pipeline
# from sklearn.decomposition import PCA
# model = make_pipeline(
#     PCA(5000),
#     SVC(kernel='linear', C=2.0)
# )
# model.fit(X_train, y_train)
# print(model.score(X_test, y_test)) #with in 2 min  refine pca   0.654188948307

## <font color="black">Nearest Neighbours</font>

In [34]:
# from sklearn.neighbors import KNeighborsClassifier
# model = KNeighborsClassifier(n_neighbors=10)
# model.fit(X_train, y_train)
# print(model.score(X_test, y_test)) #4 min   0.609625668449 = neighbor5
# #3.40 min   0.600713012478   meighbors=10         0.616755793226

## <font color="black">Jeff: I think if add weather condition into X, Scaler may need use. Because the unit is much different</font>

## <font color="black">neural_network</font>

In [35]:
# from sklearn.neural_network import MLPClassifier
# model = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(), random_state=0)
# model.fit(X_train, y_train)

# print(model.score(X_test, y_test))   # 1 min 0.393939393939

In [36]:
# from sklearn.neural_network import MLPClassifier
# model = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(10, 30),
#                       activation='logistic', random_state=0)
# model.fit(X_train, y_train)
# print(model.score(X_test, y_test)) #  1 min 0.399286987522


## <font color="black">add weather condition and time condition to X</font>

In [37]:

from sklearn.svm import SVC
X_condition = merged_data.drop(['Weather'], axis=1)
X_condition = X_condition.drop(['Time'], axis=1)
X_condition = X_condition.drop(['Date/Time','Year','Month','Day'], axis=1)
y_condition = merged_data['Weather']
X_condition_train, X_condition_test, y_condition_train, y_condition_test = train_test_split(X_condition, y_condition)


from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
model = make_pipeline(
    PCA(5000),
    SVC(kernel='linear', C=2.0)
)
model.fit(X_condition_train, y_condition_train)
print(model.score(X_condition_test, y_condition_test))   #0.704    best now

0.680926916221


## <font color="black">try deal with problem with multilable (same training dataset and test dataset as last cell)</font>

In [38]:
#change the type of Weather column
def change_str_to_array(s):
    return s.split(',')
# change y_train and y_test into proper shape
from sklearn.preprocessing import MultiLabelBinarizer
y_condition = MultiLabelBinarizer().fit_transform(merged_data['Weather'].apply(change_str_to_array))
X_condition_train, X_condition_test, y_condition_train, y_condition_test = train_test_split(X_condition, y_condition)
from sklearn.multiclass import OneVsRestClassifier
model = make_pipeline(
    PCA(5000),
    OneVsRestClassifier(SVC(kernel='linear', C=2.0))
)
model.fit(X_condition_train, y_condition_train)
print(model.score(X_condition_test, y_condition_test))  #57% with multilable, <<70.4. maybe because most row are single lablled.

0.573975044563


In [39]:
# print(X_condition_test.shape)
# print(y_condition_test.shape)
# print(X_condition_train.shape)
# print(y_condition_train.shape)
# # To: figure out why 9 columns

(561, 147463)
(561, 5)
(1683, 147463)
(1683, 5)


## <font color="blue"> IV. PRESENT RESULT </font>

## <font color="blue"> V. Other Interesting RESULT </font>

#using image to predict time of the day

In [40]:
# images_df

In [41]:
#generate training set and test set
X = images_df.iloc[:,:-4]
y = images_df['Time']
X_train, X_test, y_train, y_test = train_test_split(X, y)


In [42]:
# #trying to use SVC model 
# model = SVC(kernel='linear')
# model.fit(X_train, y_train)
# print(model.score(X_test, y_test))  #0.46, so about half can be perdicated correctly. because things like 10am,11am probably looks very similar on the picture. 

In [43]:
#trying to use SVC model with PCA
model = make_pipeline(
    PCA(5000),
    SVC(kernel='linear', C=2.0)
)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))   
# 0.5048  good about a half. and very very time consuming about 10 mins

0.504754358162
